<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/datacollection_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
1. pickle 로드
2. 로드된 df에서 마지막 수집날짜 확인
3. (마지막 수집날짜+1 ~ 오늘) 데이터 수집 (웹에서 불러옴)
4. 기존 df와 3.의 df 병합 (concat)
5. 새로운 df 다시 pickle 저장

(매일 실행)
'''

In [1]:
import re
import json
import requests
import pandas as pd
from io import BytesIO
from datetime import datetime 
from datetime import timedelta
import pickle
from google.colab import drive

drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/[Shared]K-Project/'
output_path = '/content/drive/MyDrive/k-project/output/'
# pickle 경로를 설정하기 위해

Mounted at /content/drive


In [2]:
# 한국시간 설정
import pytz

KST = pytz.timezone('Asia/Seoul')
today = str(datetime.now(KST).strftime("%Y%m%d"))
print(today)

20211003


In [3]:
## Load pickle - fullcode 목록
with open(data_path + 'full_code.pkl',"rb") as f: 
    fullcode = pickle.load(f)
fullcode

,full_code,short_code,codeName,marketEngName,status
0,KR7060310000,060310,3S,KOSDAQ,상장
1,KR7095570008,095570,AJ네트웍스,KOSPI,상장
2,KR7006840003,006840,AK홀딩스,KOSPI,상장
3,KR7054620000,054620,APS홀딩스,KOSDAQ,상장
4,KR7265520007,265520,AP시스템,KOSDAQ,상장
...,...,...,...,...,...
2580,KR7000542001,000547,흥국화재2우B,KOSPI,상장
2581,KR7000541003,000545,흥국화재우,KOSPI,상장
2582,KR7003280005,003280,흥아해운,KOSPI,상장
2583,KR7037440005,037440,희림,KOSDAQ,상장


In [4]:
# 상장폐지 상태 제외시키기
fullcode_filter = fullcode[fullcode['status'] != '상장폐지']
fullcode_filter

,full_code,short_code,codeName,marketEngName,status
0,KR7060310000,060310,3S,KOSDAQ,상장
1,KR7095570008,095570,AJ네트웍스,KOSPI,상장
2,KR7006840003,006840,AK홀딩스,KOSPI,상장
3,KR7054620000,054620,APS홀딩스,KOSDAQ,상장
4,KR7265520007,265520,AP시스템,KOSDAQ,상장
...,...,...,...,...,...
2580,KR7000542001,000547,흥국화재2우B,KOSPI,상장
2581,KR7000541003,000545,흥국화재우,KOSPI,상장
2582,KR7003280005,003280,흥아해운,KOSPI,상장
2583,KR7037440005,037440,희림,KOSDAQ,상장


In [10]:
fullcode_list = list(fullcode_filter['full_code'])
print(fullcode_list[:10])
# fullcode열을 list로 변경 후 확인

['KR7060310000', 'KR7095570008', 'KR7006840003', 'KR7054620000', 'KR7265520007', 'KR7211270004', 'KR7027410000', 'KR7282330000', 'KR7138930003', 'KR7001460005']


In [6]:
## Load pickle
# pickle로 저장되었던 데이터 출력 (어제까지의 데이터)
with open(output_path + "3S.pickle","rb") as fr:
    s3 = pickle.load(fr)
print(s3)

          TRD_DD  ISU_CD  ISU_NM TDD_CLSPRC  ...  BPS   PBR DPS DVD_YLD
0     2021/10/01  060310      3S      3,960  ...  792  5.00   0    0.00
1     2021/09/30  060310      3S      4,105  ...  792  5.18   0    0.00
2     2021/09/29  060310      3S      4,160  ...  792  5.25   0    0.00
3     2021/09/28  060310      3S      3,985  ...  792  5.03   0    0.00
4     2021/09/27  060310      3S      4,115  ...  792  5.20   0    0.00
...          ...     ...     ...        ...  ...  ...   ...  ..     ...
4804  2002/04/29  060310  삼에스코리아      6,780  ...    -     -   -       -
4805  2002/04/26  060310  삼에스코리아      7,700  ...    -     -   -       -
4806  2002/04/25  060310  삼에스코리아      8,750  ...    -     -   -       -
4807  2002/04/24  060310  삼에스코리아      8,510  ...    -     -   -       -
4808  2002/04/23  060310  삼에스코리아      7,600  ...    -     -   -       -

[4809 rows x 14 columns]


In [8]:
# 매일 pickle 데이터 불러와서 읽은 후 0행, 0열의 날짜 확인
# 초기 데이터가 없는 데이터? -> if문 or try~except 예외처리

date = s3.iloc[0,0]
yesterday = re.sub('/','-',date)
yesterday = datetime.strptime(yesterday, '%Y-%m-%d') + timedelta(days = 1)
yesterday = yesterday.strftime("%Y%m%d")
print(yesterday)

20211002


In [15]:
strtDd = '20211001' #yesterday
endDd = today #20211003

url2 = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
headers = {'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',}

data2 = { 'bld': 'dbms/MDC/STAT/issue/MDCSTAT23902',
    'isuCd': fullcode_list[0],
    'isuCd2': '',
    'strtDd': strtDd,
    'endDd': endDd,
    'share': '1',
    'money': '1',
    'csvxls_isNo': 'false',}

r2 = requests.post(url2, data2, headers)
j2 = json.loads(r2.text)
df2=pd.DataFrame(j2['output'])
# print(df2)
# print('------------------------------------------------')

data3 = { 'bld': 'dbms/MDC/STAT/standard/MDCSTAT03502',
    'isuCd': fullcode_list[0],
    'isuCd2': fullcode_list[0],
    'strtDd': strtDd,
    'endDd': endDd,
    'searchType': '2',
    'mktId': 'ALL',
    'csvxls_isNo': 'false',}

r2 = requests.post(url2, data3, headers)
j2 = json.loads(r2.text)
df3=pd.DataFrame(j2['output'])
# print(df3)
# print('------------------------------------------------')

df2_ = df2[['TRD_DD', 'ISU_CD', 'ISU_NM', 'TDD_CLSPRC', 'TDD_OPNPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'MKTCAP']]
df3_ = df3[['TRD_DD', 'EPS', 'PER', 'BPS', 'PBR', 'DPS', 'DVD_YLD']]

result = pd.merge(df2_, df3_)
print(result)
print('------------------------------------------------')

       TRD_DD  ISU_CD ISU_NM TDD_CLSPRC  ...  BPS   PBR DPS DVD_YLD
0  2021/10/01  060310     3S      3,960  ...  792  5.00   0    0.00

[1 rows x 14 columns]
------------------------------------------------


In [17]:
output = pd.concat([result, s3], ignore_index = True)
print(output)

          TRD_DD  ISU_CD  ISU_NM TDD_CLSPRC  ...  BPS   PBR DPS DVD_YLD
0     2021/10/01  060310      3S      3,960  ...  792  5.00   0    0.00
1     2021/10/01  060310      3S      3,960  ...  792  5.00   0    0.00
2     2021/09/30  060310      3S      4,105  ...  792  5.18   0    0.00
3     2021/09/29  060310      3S      4,160  ...  792  5.25   0    0.00
4     2021/09/28  060310      3S      3,985  ...  792  5.03   0    0.00
...          ...     ...     ...        ...  ...  ...   ...  ..     ...
4805  2002/04/29  060310  삼에스코리아      6,780  ...    -     -   -       -
4806  2002/04/26  060310  삼에스코리아      7,700  ...    -     -   -       -
4807  2002/04/25  060310  삼에스코리아      8,750  ...    -     -   -       -
4808  2002/04/24  060310  삼에스코리아      8,510  ...    -     -   -       -
4809  2002/04/23  060310  삼에스코리아      7,600  ...    -     -   -       -

[4810 rows x 14 columns]


In [18]:
## Save pickle (업데이트된 데이터프레임 다시 저장)
with open(output_path + "3S.pickle","w+b") as fw:
    pickle.dump(output, fw)

In [19]:
## Load pickle
# pickle로 저장되었던 데이터 출력 (어제까지의 데이터)
with open(output_path + "3S.pickle","rb") as fr:
    s3 = pickle.load(fr)
print(s3)

          TRD_DD  ISU_CD  ISU_NM TDD_CLSPRC  ...  BPS   PBR DPS DVD_YLD
0     2021/10/01  060310      3S      3,960  ...  792  5.00   0    0.00
1     2021/10/01  060310      3S      3,960  ...  792  5.00   0    0.00
2     2021/09/30  060310      3S      4,105  ...  792  5.18   0    0.00
3     2021/09/29  060310      3S      4,160  ...  792  5.25   0    0.00
4     2021/09/28  060310      3S      3,985  ...  792  5.03   0    0.00
...          ...     ...     ...        ...  ...  ...   ...  ..     ...
4805  2002/04/29  060310  삼에스코리아      6,780  ...    -     -   -       -
4806  2002/04/26  060310  삼에스코리아      7,700  ...    -     -   -       -
4807  2002/04/25  060310  삼에스코리아      8,750  ...    -     -   -       -
4808  2002/04/24  060310  삼에스코리아      8,510  ...    -     -   -       -
4809  2002/04/23  060310  삼에스코리아      7,600  ...    -     -   -       -

[4810 rows x 14 columns]


In [ ]:
# 수집 자동화 -> schedule 라이브러리 (실현가능성?), 윈도우 작업 스케쥴러/crontab (운영체제 제한), github actions
# 다른 종목 수집?
# 최종 산출물에 필요한 컬럼?